In [31]:
from PIL import Image,ImageDraw,ImageFont
from ultralytics import YOLO
from os import walk
import easyocr
import numpy as np
import xlsxwriter
import os
 

In [6]:
model = YOLO('./weights/last.pt')

In [43]:
myFont = ImageFont.truetype('./Roboto-Bold.ttf', 30)
reader = easyocr.Reader(['en'])
mypath="./files_to_scan"
filenames = next(walk(mypath), (None, None, []))[2]  # [] if no file

output_path="./output-marked_img"
if not os.path.exists(output_path):
    os.makedirs(output_path)


workbook = xlsxwriter.Workbook('Example2.xlsx')
worksheet = workbook.add_worksheet()
 
# Start from the first cell.
# Rows and columns are zero indexed.
row = 1
column = 0
worksheet.write(0, 0, "Reference Number")
worksheet.write(0, 1, "2 Ink Seal Found yes/no")
worksheet.write(0, 2, "Original Img location")
worksheet.write(0, 3, "Marked Image Location")
worksheet.set_column(0, 3, 40)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


0

In [12]:
def is_vertically_flipped(img):
    orig=Image.open("./POD1.PNG").convert('RGB')
    tst = img.transpose(Image.ROTATE_180)
    if np.any(orig != tst):
        return img
    else:
        return tst

In [44]:
for filename in filenames:
    img_path=mypath+"/"+filename
    img = Image.open(img_path).convert('RGB')

    img=is_vertically_flipped(img)
    img.save(img_path)

    results = model.predict(img, conf=0.30)

    text_result = reader.readtext(img_path, detail = 0)

    ref_no=""
    for txts in text_result:
        if(txts.__contains__("Ref#")):
            ref_no=txts.split("#")[1].split("/")[0]

    i=0
    for det in results[0].boxes.xyxy:
        x1, y1, x2, y2 = det
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        draw = ImageDraw.Draw(img)
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
        draw2 = ImageDraw.Draw(img)
        draw2.text((x1+10, y1+10),ref_no, font=myFont,fill=(255, 0, 0),outline=(0,0,0))
        i=i+1
    draw = ImageDraw.Draw(img)
    if results[0].boxes.xyxy.size()[0]>=2:
        draw2.text((10, 10),"Picture with two ink stamps", font=myFont,fill=(255, 0, 0),outline=(0,0,0))
    else:
        draw2.text((10, 10),"Picture with less than two ink stamps", font=myFont,fill=(255, 0, 0),outline=(0,0,0))

    seal_present="Yes"
    if(results[0].boxes.xyxy.size()[0]>=2):
        seal_present="Yes"
    else:
        seal_present="No"


    img.save(output_path+"/"+ref_no+".jpg")

    worksheet.write(row, 0, ref_no)
    worksheet.write(row, 1,seal_present )
    worksheet.write(row, 2, img_path)
    worksheet.write(row, 3, output_path+"/"+ref_no+".jpg")
    row+=1
    img.show()

workbook.close()


0: 352x800 2 Ink-stampss, 236.2ms
Speed: 0.0ms preprocess, 236.2ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 800)

0: 352x800 2 Ink-stampss, 191.8ms
Speed: 3.1ms preprocess, 191.8ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 800)
